<a href="https://colab.research.google.com/github/ChintPatel/CMPE-255-Apache-Beam-Assignment/blob/main/Apache_Beam_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install apache-beam[gcp]


In [4]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
import datetime


In [9]:
import pandas as pd

# Create a sample static dataset to simulate streaming data
import random
import datetime

# Generating 100 sample records
data = [{'word': random.choice(['apple', 'banana', 'cherry', 'date', 'elderberry']),
         'timestamp': datetime.datetime.now() + datetime.timedelta(seconds=i)}
        for i in range(100)]

# Save to CSV to simulate input source
pd.DataFrame(data).to_csv('sample_data.csv', index=False)

import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
import datetime

# Define pipeline options
pipeline_options = PipelineOptions()

# Composite Transform for counting elements
class CountWords(beam.PTransform):
    def expand(self, pcoll):
        return (pcoll
                | 'PairWithOne' >> beam.Map(lambda x: (x, 1))
                | 'GroupAndSum' >> beam.CombinePerKey(sum))

# Custom `ParDo` function to extract the word field
class ExtractWord(beam.DoFn):
    def process(self, element):
        yield element['word']

# Define the Apache Beam pipeline
with beam.Pipeline(options=pipeline_options) as p:

    # 1. Input Data from CSV (Pipeline IO)
    data_stream = (p
                   | 'Read CSV' >> beam.io.ReadFromText('sample_data.csv', skip_header_lines=1)
                   | 'Parse CSV' >> beam.Map(lambda line: {'word': line.split(',')[0], 'timestamp': line.split(',')[1]})
                   | 'Extract Word' >> beam.ParDo(ExtractWord()))

    # 2. Apply Windowing with AfterWatermark
    windowed_words = (data_stream
                      | 'Apply Window' >> beam.WindowInto(
                            beam.window.FixedWindows(10),  # Fixed window of 10 seconds
                            trigger=beam.trigger.AfterWatermark(),  # Safe trigger
                            accumulation_mode=beam.trigger.AccumulationMode.DISCARDING))

    # 3. Composite Transform for Word Count
    word_counts = windowed_words | 'CountWords' >> CountWords()

    # 4. Output Results to Console
    word_counts | 'Print Results' >> beam.Map(print)


('date', 17)
('banana', 18)
('cherry', 24)
('elderberry', 22)
('apple', 19)
